# Conjunto de dados do Titanic - Vetores One-Hot

Nesta Unidade vamos construir um modelo para prever quem sobreviveu ao desastre do Titanic.

Ao fazer isso, praticaremos a transformação de dados entre tipos numéricos e categóricos, incluindo o uso de vetores one-hot.

## Preparando dados

Vamos começar abrindo e limpando rapidamente nosso conjunto de dados, como fizemos na última unidade:

In [3]:
import pandas
!python -m wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/m0c_logistic_regression.py
!python -m wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/titanic.csv

# Open our dataset from file
dataset = pandas.read_csv("titanic.csv", index_col=False, sep=",", header=0)

# Fill missing cabin information with 'Unknown'
dataset["Cabin"].fillna("Unknown", inplace=True)

# Remove rows missing Age information
dataset.dropna(subset=["Age"], inplace=True)

# Remove the Name, PassengerId, and Ticket fields
# This is optional and only to make it easier to read our print-outs
dataset.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace=True)

dataset.head()


Saved under m0c_logistic_regression.py

Saved under titanic (2).csv


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,Unknown,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,Unknown,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,Unknown,S


## Sobre nosso modelo

Estaremos treinando um tipo de modelo chamado Regressão Logística, que irá prever quem sobreviverá ao desastre do Titanic.

Você não precisa entender de regressão logística para entender este exercício, então colocamos a implementação de fora deste notebook em um método chamado train_logistic_regression. Se você estiver curioso, pode ler este método em nosso repositório GitHub.

train_logistic_regression:

 1. Aceita nosso quadro de dados e uma lista de recursos a serem incluídos no modelo.
 2. Treina o modelo
 3. Retorna um número informando o desempenho do modelo na previsão de sobrevivência. Números menores são melhores.

## Apenas Numérico

Vamos criar um modelo, usando apenas os recursos numéricos.

Primeiro, usaremos Pclass aqui como um recurso ordinal, em vez de um recurso categórico codificado one-hot.

In [4]:
from m0c_logistic_regression import train_logistic_regression

features = ["Age", "Pclass", "SibSp", "Parch", "Fare"] 
loss_numerical_only = train_logistic_regression(dataset, features)

print(f"Numerical-Only, Log-Loss (cost): {loss_numerical_only}")

Numerical-Only, Log-Loss (cost): 0.6121682789483456


Temos nosso ponto de partida. Vamos ver se podemos melhorar o modelo usando recursos categóricos.

## Recursos categóricos binários

Os recursos categóricos que têm apenas dois valores potenciais podem ser codificados em uma única coluna como 0 e 1.

Vamos converter os Sex valores em IsFemale- a 0 para masculino e 1 feminino - e incluir isso em nosso modelo.

In [5]:
# Swap male / female with numerical values
# We can do this because there are only two categories
dataset["IsFemale"] = dataset.Sex.replace({'male':0, 'female':1})

# Print out the first few rows of the dataset
print(dataset.head())

# Run and test the model, also using IsFemale this time
features = ["Age", "Pclass", "SibSp", "Parch", "Fare", "IsFemale"] 
loss_binary_categoricals = train_logistic_regression(dataset, features)

print(f"\nNumerical + Sex, Log-Loss (cost): {loss_binary_categoricals}")

   Survived  Pclass     Sex   Age  SibSp  Parch     Fare    Cabin Embarked  \
0         0       3    male  22.0      1      0   7.2500  Unknown        S   
1         1       1  female  38.0      1      0  71.2833      C85        C   
2         1       3  female  26.0      0      0   7.9250  Unknown        S   
3         1       1  female  35.0      1      0  53.1000     C123        S   
4         0       3    male  35.0      0      0   8.0500  Unknown        S   

   IsFemale  
0         0  
1         1  
2         1  
3         1  
4         0  

Numerical + Sex, Log-Loss (cost): 0.47071447414358547


Nossa perda (erro) diminuiu! Isso significa que este modelo tem um desempenho melhor do que o anterior.

## Codificação One-Hot

A classe de ticket ( Pclass) é um recurso Ordinal. Isso significa que seus valores potenciais (1, 2 e 3) são tratados como tendo uma ordem e igualmente espaçados. É possível que esse espaçamento uniforme simplesmente não esteja correto - nas histórias que ouvimos sobre o Titanic, os passageiros da terceira classe foram tratados muito pior do que os da 1ª e 2ª classe.

Vamos converter Pclassem um recurso categórico usando a codificação one-hot:

In [6]:
# Get all possible categories for the "PClass" column
print(f"Possible values for PClass: {dataset['Pclass'].unique()}")

# Use Pandas to One-Hot encode the PClass category
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass"], drop_first=False)

# Add back in the old Pclass column, for learning purposes
dataset_with_one_hot["Pclass"] = dataset.Pclass

# Print out the first few rows
dataset_with_one_hot.head()

Possible values for PClass: [3 1 2]


,Survived,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,IsFemale,Pclass_1,Pclass_2,Pclass_3,Pclass
0,0,male,22.0,1,0,7.2500,Unknown,S,0,0,0,1,3
1,1,female,38.0,1,0,71.2833,C85,C,1,1,0,0,1
2,1,female,26.0,0,0,7.9250,Unknown,S,1,0,0,1,3
3,1,female,35.0,1,0,53.1000,C123,S,1,1,0,0,1
4,0,male,35.0,0,0,8.0500,Unknown,S,0,0,0,1,3


Veja como Pclass foi convertido em três valores: Pclass_1, Pclass_2 e Pclass_3.

Linhas com Pclass1 têm um valor na Pclass_1 coluna. O mesmo padrão existe para valores de 2 e 3.

Vamos agora executar novamente nosso modelo tratando os Pclass valores como categóricos, em vez de ordinais.

In [7]:
# Run and test the model, also using Pclass as a categorical feature this time
features = ["Age", "SibSp", "Parch", "Fare", "IsFemale",
            "Pclass_1", "Pclass_2", "Pclass_3"]

loss_pclass_categorical = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nNumerical, Sex, Categorical Pclass, Log-Loss (cost): {loss_pclass_categorical}")


Numerical, Sex, Categorical Pclass, Log-Loss (cost): 0.471711186710776


Isso parece ter piorado um pouco as coisas!

Vamos continuar.

## Incluindo Cabine

Lembre-se que muitos passageiros tinham Cabin informações. Cabin é uma característica categórica e deve ser um bom preditor de sobrevivência, porque as pessoas em cabines mais baixas provavelmente tiveram pouco tempo para escapar durante o naufrágio.

Vamos codificar a cabine usando vetores one-hot e incluí-la em um modelo. Há tantas cabines desta vez que não vamos imprimir todas. Se você quiser praticar imprimi-los, sinta-se à vontade para editar o código como prática.

In [8]:
# Use Pandas to One-Hot encode the Cabin and Pclass categories
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass", "Cabin"], drop_first=False)

# Find cabin column names
cabin_column_names = list(c for c in dataset_with_one_hot.columns if c.startswith("Cabin_"))

# Print out how many cabins there were
print(len(cabin_column_names), "cabins found")

# Make a list of features
features = ["Age", "SibSp", "Parch", "Fare", "IsFemale",
            "Pclass_1", "Pclass_2", "Pclass_3"] + \
            cabin_column_names

# Run the model and print the result
loss_cabin_categorical = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nNumerical, Sex, Categorical Pclass, Cabin, Log-Loss (cost): {loss_cabin_categorical}")

135 cabins found

Numerical, Sex, Categorical Pclass, Cabin, Log-Loss (cost): 0.46007129627873466


Esse é o nosso melhor resultado até agora!

## Melhorando o Poder

Incluir um número muito grande de classes categóricas - como 135 Cabines - geralmente não é a melhor maneira de treinar um modelo. Isso ocorre porque o modelo tem apenas alguns exemplos de cada classe de categoria para aprender.

Às vezes, os modelos podem ser aprimorados simplificando os recursos. Cabin provavelmente era útil porque indicava em qual andar do povo titânico provavelmente estava situado: aqueles nos conveses inferiores teriam seus aposentos inundados primeiro.

Usar as informações do deck pode ser mais simples do que categorizar as pessoas em Cabines.

Vamos simplificar o que rodamos, substituindo as 135 Cabin categorias por uma categoria mais simples Deck, que possui apenas 9 valores: A - G, T e U (Desconhecido)

In [9]:
# We have cabin names, like A31, G45. The letter refers to the deck that
# the cabin was on. Extract just the deck and save it to a column. 
dataset["Deck"] = [c[0] for c in dataset.Cabin]

print("Decks: ", sorted(dataset.Deck.unique()))

# Create one-hot vectors for:
# Pclass - the class of ticket. (This could be treated as ordinal or categorical)
# Deck - the deck that the cabin was on
dataset_with_one_hot = pandas.get_dummies(dataset, columns=["Pclass", "Deck"], drop_first=False)

# Find the deck names
deck_of_cabin_column_names = list(c for c in dataset_with_one_hot.columns if c.startswith("Deck_"))
 
features = ["Age", "IsFemale", "SibSp", "Parch", "Fare", 
            "Pclass_1", "Pclass_2", "Pclass_3",
            "Deck_A", "Deck_B", "Deck_C", "Deck_D", 
            "Deck_E", "Deck_F", "Deck_G", "Deck_U", "Deck_T"]

loss_deck = train_logistic_regression(dataset_with_one_hot, features)

print(f"\nSimplifying Cabin Into Deck, Log-Loss (cost): {loss_deck}")

Decks:  ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'U']

Simplifying Cabin Into Deck, Log-Loss (cost): 0.45884220743082615


## Comparando modelos

Vamos comparar o loss para estes modelos:

In [10]:
# Use a dataframe to create a comparison table of metrics
# Copy metrics from previous Unit

l =[["Numeric Features Only", loss_numerical_only],
    ["Adding Sex as Binary", loss_binary_categoricals],
    ["Treating Pclass as Categorical", loss_pclass_categorical],
    ["Using Cabin as Categorical", loss_cabin_categorical],
    ["Using Deck rather than Cabin", loss_deck]]

pandas.DataFrame(l, columns=["Dataset", "Log-Loss (Low is better)"])

,Dataset,Log-Loss (Low is better)
0,Numeric Features Only,0.612168
1,Adding Sex as Binary,0.470714
2,Treating Pclass as Categorical,0.471711
3,Using Cabin as Categorical,0.460071
4,Using Deck rather than Cabin,0.458842


Podemos ver que incluir recursos categóricos pode melhorar e prejudicar o funcionamento de um modelo. Muitas vezes, a experimentação é a melhor maneira de encontrar o melhor modelo.

# Resumo

Nesta unidade você aprendeu como usar a codificação One-Hot para endereçar dados categóricos.

Também exploramos como, às vezes, pensar criticamente sobre o problema que você está tentando pode melhorar uma solução melhor do que simplesmente incluir todos os recursos possíveis em um modelo.